In [6]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import tensorflow as tf

import model as md
import utils as ut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# load example inputs and outputs
inFilePath = "/asic/projects/C/CMS_PIX_28/pixelAV_datasets/unshuffled_DO_NOT_DELETE/initial_studies/dataset14/unflipped"
yprofiles, ylocals, clslabels = ut.loadParquetData(inFilePath=inFilePath, qm_charge_levels = [400, 1600, 2400], qm_quant_values = [0, 1, 2, 3])
print(yprofiles.shape, ylocals.shape, clslabels.shape)

# assign to X, Y
X = yprofiles
Y = clslabels 
print(X.shape, Y.shape)

100%|██████████| 72/72 [02:50<00:00,  2.37s/it]


3597877 3597877
Creating yprofiles
(3597877, 16) (3597877,) (3597877,)
(3597877, 16) (3597877,)


In [10]:
# create model
shape = 16 # y-profile ... why is this 16 and not 8?
nb_classes = 3 # positive low pt, negative low pt, high pt
first_dense = 58 # shape of first dense layer
outDir = "./tmp"

# keras model
model_file = "/fasic_home/gdg/research/projects/CMS_PIX_28/directional-pixel-detectors/multiclassifier/models/ds8l6_padded_noscaling_keras_d58model.h5"
model = md.CreateModel(shape, nb_classes, first_dense, model_file = model_file)

# qkeras model
qmodel_file = "/fasic_home/gdg/research/projects/CMS_PIX_28/directional-pixel-detectors/multiclassifier/models/ds8l6_padded_noscaling_qkeras_foldbatchnorm_d58w4a8model.h5"
qmodel = md.CreateQModel(shape, model_file=qmodel_file)
# generate hls model
ut.gen_hls_model(qmodel, output_dir=outDir)
# prepare weights for ASIC
ut.prepareWeights(os.path.join(outDir, "firmware/weights/"))

Model: "model_912"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16)]              0         
                                                                 
 dense1 (Dense)              (None, 58)                986       
                                                                 
 batch_normalization_456 (B  (None, 58)                232       
 atchNormalization)                                              
                                                                 
 relu1 (Activation)          (None, 58)                0         
                                                                 
 dense2 (Dense)              (None, 3)                 177       
                                                                 
 linear (Activation)         (None, 3)                 0         
                                                         

/asic/projects/C/CMS_PIX_28/testing/tools/venvs/p3.11.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


'./tmp/firmware/weights/b5_w5_b2_w2_pixel_bin.csv'

In [11]:
# Compute loss and accuracy manually
def getLA(y, predictions, loss_fn, acc_metric=tf.keras.metrics.SparseCategoricalAccuracy()):
    loss = loss_fn(y, predictions).numpy()
    acc_metric.update_state(y, predictions)
    accuracy = acc_metric.result().numpy()
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy}")
    return loss, accuracy

# evaluating
verbose = 1
batch_size = 2048

# make predictions
predictions = model.predict(X, batch_size = batch_size, verbose=verbose)
# predictions = np.argmax(predictions, axis=1)
np.save(os.path.join(outDir, f"keras_predictions.npy"), predictions)
model_loss, model_acc = getLA(Y, predictions, md.custom_loss_function)

# make predictions
qpredictions = qmodel.predict(X, batch_size = batch_size, verbose=verbose)
# predictions = np.argmax(predictions, axis=1)
np.save(os.path.join(outDir, f"qkeras_predictions.npy"), qpredictions)
qmodel_loss, qmodel_acc = getLA(Y, qpredictions, md.custom_loss_function)



1757/1757 [==============================] - 2s 1ms/step
Test loss: 2.291407585144043
Test accuracy: 0.33843153715133667
1757/1757 [==============================] - 3s 1ms/step
Test loss: 1.6315134763717651
Test accuracy: 0.33808133006095886
